Calculates the subscriber count, average open rate, and average click rate for each email list. It filters out incomplete records and sorts the lists by average open rate in descending order.

In [0]:
%sql
SELECT
    dl.list_name,
    COUNT(DISTINCT f.unique_email_id) AS subscribers,
    AVG(f.stats_avg_open_rate) AS avg_open_rate,
    AVG(f.stats_avg_click_rate) AS avg_click_rate
FROM delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/fact_list_membership` f
JOIN delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/dim_list` dl
  ON f.list_id = dl.list_id
WHERE f.stats_avg_open_rate IS NOT NULL
  AND f.stats_avg_click_rate IS NOT NULL
  AND f.status = 'subscribed'
GROUP BY dl.list_name
ORDER BY avg_open_rate DESC


retrieves the top 10 subscribed contacts with over 5 list memberships, calculating each contact's list count and open rate standard deviation, and orders them by the highest variability.

In [0]:
%sql
SELECT 
    f.unique_email_id,
    dc.full_name,
    COUNT(DISTINCT f.list_id) AS list_count,
    STDDEV(f.stats_avg_open_rate) AS open_rate_stddev
FROM delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/fact_list_membership` f
JOIN delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/dim_contact` dc
  ON f.unique_email_id = dc.unique_email_id
WHERE f.status = 'subscribed'
  AND f.stats_avg_open_rate IS NOT NULL
GROUP BY f.unique_email_id, dc.full_name
HAVING COUNT(DISTINCT f.list_id) > 5
ORDER BY open_rate_stddev DESC
LIMIT 10


counts monthly signups for each email list by grouping on list name and the month of signup, including only subscribed users with a valid signup timestamp. The results are then ordered by list name and month.

In [0]:
%sql
SELECT
    dl.list_name,
    date_trunc('month', f.timestamp_signup) AS signup_month,
    COUNT(*) AS monthly_signups
FROM delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/fact_list_membership` f
JOIN delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/dim_list` dl
  ON f.list_id = dl.list_id
WHERE f.status = 'subscribed'
  AND f.timestamp_signup IS NOT NULL
GROUP BY dl.list_name, date_trunc('month', f.timestamp_signup)
ORDER BY dl.list_name, signup_month


 counts monthly signups for each email list using either the signup or opt-in timestamp. It groups and orders the results by list name and month.

In [0]:
%sql
SELECT
    dl.list_name,
    date_trunc('month', COALESCE(f.timestamp_signup, f.timestamp_opt)) AS signup_month,
    COUNT(*) AS monthly_signups
FROM delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/fact_list_membership` f
JOIN delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/dim_list` dl
  ON f.list_id = dl.list_id
WHERE f.status = 'subscribed'
  AND (f.timestamp_signup IS NOT NULL OR f.timestamp_opt IS NOT NULL)
GROUP BY dl.list_name, date_trunc('month', COALESCE(f.timestamp_signup, f.timestamp_opt))
ORDER BY dl.list_name, signup_month


Monthly Signups for Top 5 Most Subscribed Lists

In [0]:
import pandas as pd
import plotly.express as px

# Query to get monthly signups for all lists using COALESCE on timestamps
monthly_signups_df = spark.sql("""
SELECT
    dl.list_name,
    date_trunc('month', COALESCE(f.timestamp_signup, f.timestamp_opt)) AS signup_month,
    COUNT(*) AS monthly_signups
FROM delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/fact_list_membership` f
JOIN delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/dim_list` dl
  ON f.list_id = dl.list_id
WHERE f.status = 'subscribed'
  AND (f.timestamp_signup IS NOT NULL OR f.timestamp_opt IS NOT NULL)
GROUP BY dl.list_name, date_trunc('month', COALESCE(f.timestamp_signup, f.timestamp_opt))
ORDER BY dl.list_name, signup_month
""")

# Query to determine the top 5 most subscribed lists by counting distinct subscribed contacts
top_lists_df = spark.sql("""
SELECT 
    dl.list_name, 
    COUNT(DISTINCT f.unique_email_id) AS num_subscribers
FROM delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/fact_list_membership` f
JOIN delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/dim_list` dl
  ON f.list_id = dl.list_id
WHERE f.status = 'subscribed'
  AND (f.timestamp_signup IS NOT NULL OR f.timestamp_opt IS NOT NULL)
GROUP BY dl.list_name
ORDER BY num_subscribers DESC
LIMIT 5
""")

# Convert top lists to Pandas to get the list names
top_lists_pd = top_lists_df.toPandas()
top5_lists = list(top_lists_pd['list_name'])

# Filter the monthly signups DataFrame to only include the top 5 lists
monthly_top5_df = monthly_signups_df.filter(monthly_signups_df.list_name.isin(top5_lists))

# Convert to Pandas for plotting
monthly_top5_pd = monthly_top5_df.toPandas()

# Convert the signup_month column to datetime
monthly_top5_pd['signup_month'] = pd.to_datetime(monthly_top5_pd['signup_month'])

# Create an interactive line plot using Plotly Express
fig = px.line(
    monthly_top5_pd, 
    x='signup_month', 
    y='monthly_signups', 
    color='list_name',
    markers=True,
    title="Monthly Signups for Top 5 Most Subscribed Lists"
)

# Update layout with axis titles
fig.update_layout(
    xaxis_title="Signup Month",
    yaxis_title="Monthly Signups",
    legend_title="List Name"
)

# Show the plot
fig.show()


Displays the distribution of contacts by language from your dim_contact table. It can help you understand the diversity of your audience.

In [0]:
import pandas as pd
import plotly.graph_objects as goa

# Query to compute average engagement metrics per list
engagement_df = spark.sql("""
SELECT
    dl.list_name,
    AVG(f.stats_avg_open_rate) AS avg_open_rate,
    AVG(f.stats_avg_click_rate) AS avg_click_rate
FROM delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/fact_list_membership` f
JOIN delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/dim_list` dl
  ON f.list_id = dl.list_id
WHERE f.status = 'subscribed'
  AND f.stats_avg_open_rate IS NOT NULL
  AND f.stats_avg_click_rate IS NOT NULL
GROUP BY dl.list_name
ORDER BY avg_open_rate DESC
""")

# Convert to Pandas DataFrame
engagement_pd = engagement_df.toPandas()

# Create a grouped bar chart using Plotly
fig1 = go.Figure(data=[
    go.Bar(name='Avg Open Rate', x=engagement_pd['list_name'], y=engagement_pd['avg_open_rate']),
    go.Bar(name='Avg Click Rate', x=engagement_pd['list_name'], y=engagement_pd['avg_click_rate'])
])

fig1.update_layout(
    barmode='group',
    title='Average Engagement Metrics per List',
    xaxis_title='List Name',
    yaxis_title='Rate',
    legend_title='Metric'
)

fig1.show()


shows each list's engagement by plotting its average open rate versus its average click rate. The size of each point represents the number of subscribers, and each list is identified by color and label.

In [0]:
import pandas as pd
import plotly.express as px

# Query to get per-list engagement metrics: average open rate, average click rate, and subscriber count
list_metrics_df = spark.sql("""
SELECT
    dl.list_name,
    AVG(f.stats_avg_open_rate) AS avg_open_rate,
    AVG(f.stats_avg_click_rate) AS avg_click_rate,
    COUNT(DISTINCT f.unique_email_id) AS subscribers
FROM delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/fact_list_membership` f
JOIN delta.`abfss://gold@mailchimpspnetwork.dfs.core.windows.net/dim_list` dl
  ON f.list_id = dl.list_id
WHERE f.status = 'subscribed'
  AND f.stats_avg_open_rate IS NOT NULL
  AND f.stats_avg_click_rate IS NOT NULL
GROUP BY dl.list_name
ORDER BY subscribers DESC
""")

# Convert the results to a Pandas DataFrame
list_metrics_pd = list_metrics_df.toPandas()

# Create an interactive scatter plot using Plotly Express
fig3 = px.scatter(
    list_metrics_pd,
    x='avg_open_rate',
    y='avg_click_rate',
    size='subscribers',
    color='list_name',
    hover_name='list_name',
    title='List Engagement: Open Rate vs Click Rate',
    labels={
        'avg_open_rate': 'Average Open Rate',
        'avg_click_rate': 'Average Click Rate',
        'subscribers': 'Number of Subscribers'
    }
)

fig3.show()
